In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
import time
import itertools
from datetime import datetime
from urllib.parse import urlparse,urlsplit,parse_qs,parse_qsl,urlencode,urlunparse


# TUN AIRPORT

In [2]:
def airport_TUN(type_='departures'):
    list_ = []
    rows=[]
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~]')

    if type_=='departures':
        html = requests.get("http://www.aeroport-de-tunis-carthage.com/tunisie-aeroport-de-tunis-carthage-vols-depart.php").text
    elif type_=='arrivals': 
        html = requests.get("http://www.aeroport-de-tunis-carthage.com/tunisie-aeroport-de-tunis-carthage-vols-arrivee.php").text

    soup = BeautifulSoup(html, "html.parser")
    
    i=0
    for row in soup.find_all('tr'):
        temp_cols = row.find_all('td')
        if len(temp_cols) > 1 and ":" in temp_cols[0].text and not temp_cols[0].table and row.get_text():
            rows.append(row.text)  
    for i in range(len(rows)):
        x = re.split("\n",rows[i])
        x = re.split("\n\n",rows[i])
        i=0
        x = list(filter(None, x))
        for i in range(len(x)):
            if  regex.search(x[i]) == None:
                    b = re.split("\n",x[i])
                    b = list(filter(None, b))                    
                    list_.append(b)  
    list_.sort()
    list_ = list(list_ for list_,_ in (itertools.groupby(list_))) 
    if len(list_[0])== 1:
        del list_[0]
    if list_[len(list_)-1][1] in {'Destination','Origine','Origin'}:
        list_.pop(len(list_)-1)
    list_ = [{'time':datetime.strptime(i[0],'%H:%M:%S').time(),'destination':i[1],'airline':i[2],'flight':i[3],'status':i[4]} for i in list_]
    for i in list_:
        if ":" in i.get('status',''):
            status,status_time = i.get('status').split(' ')
            i.update({'status':status,'status_time':datetime.strptime(status_time,'%H:%M').time()})
    return list_

In [3]:
x = airport_TUN(type_='arrivals')

In [4]:
x

[{'time': datetime.time(0, 20),
  'destination': 'FRANCFORT',
  'airline': 'LUFTHANSA',
  'flight': 'LH1326',
  'status': 'ATTERRI',
  'status_time': datetime.time(0, 21)},
 {'time': datetime.time(1, 40),
  'destination': 'MADRID',
  'airline': 'AIR EUROPA',
  'flight': 'UX1115',
  'status': 'ATTERRI',
  'status_time': datetime.time(1, 48)},
 {'time': datetime.time(4, 0),
  'destination': 'CONAKRY',
  'airline': 'TUNISAIR',
  'flight': 'TU841',
  'status': 'ATTERRI',
  'status_time': datetime.time(4, 34)},
 {'time': datetime.time(4, 50),
  'destination': 'NIAMEY AD',
  'airline': 'TUNISAIR',
  'flight': 'TU399',
  'status': 'ATTERRI',
  'status_time': datetime.time(4, 20)},
 {'time': datetime.time(5, 35),
  'destination': 'NOUAKCHOTT',
  'airline': 'TUNISAIR',
  'flight': 'TU564',
  'status': 'ATTERRI',
  'status_time': datetime.time(6, 7)},
 {'time': datetime.time(7, 45),
  'destination': 'JEDDAH',
  'airline': 'TUNISAIR',
  'flight': 'TU714',
  'status': 'ATTERRI',
  'status_time': d

# TUN OACA

In [7]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
import time
import itertools
from datetime import datetime
from urllib.parse import urlparse,urlsplit,parse_qs,parse_qsl,urlencode,urlunparse

def airport_TUN(type_='departures',date=''):
    list_ = []
    rows=[]
    regex = re.compile('[@_!#$%^&*()<>?/\|}{~]')
    airports=['tunis','tozeur','djerba','tabarka','sfax','gafsa','gabes']
    #Date Format is YEAR-MONTH-DAY
    if date == '':
        currentDay = datetime.today().day
        currentMonth = datetime.today().month
        currentYear = datetime.today().year
    else:
        date = date.split('-')
        currentYear = date[0]
        currentMonth = date[1]
        currentDay = date[2]
        
        
        
    if type_=='departures':
        url = urlparse('http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmnumVol=&frmairport=tunis&frmday=7&frmmonth=10&frmacty=2019&frmhour=0')
    elif type_=='arrivals': 
        url = urlparse('http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=A&frmaeropVil=-1&frmnumVol=&frmairport=tunis&frmday=7&frmmonth=10&frmacty=2019&frmhour=0')
    

    
    for airport in airports:


        
        params = {'frmairport':airport,'frmmonth': currentMonth,'frmacty':currentYear ,'frmday':currentDay }
        url_parts = list(url)
        query = dict(parse_qsl(url_parts[4]))
        query.update(params)
        url_parts[4] = urlencode(query)
        

        request = urlunparse(url_parts)
 

        print (request)
        html = requests.get(request).text

        soup = BeautifulSoup(html, "html.parser")

        i=0
        for row in soup.find_all('tr'):
            temp_cols = row.find_all('td')
            rows.append(row.text)  
        for i in range(len(rows)):
            x = re.split("\n",rows[i])
            x = re.split("\n\n",rows[i])
            i=0
            x = list(filter(None, x))
            for num,i in enumerate(range(len(x))):
                #print(x[i])
                if  regex.search(x[i]) == None:
                        b = re.split("\n",x[i])
                        #print(b)
                        b = list(filter(None, b))
                        if '\xa0\r' in b or not(b) or 'Heure' in b :
                            pass
                        else:
                            list_.append(b)
    list_.sort()
    list_ = list(list_ for list_,_ in (itertools.groupby(list_))) 
    list_ = [{'Origine':i[0],'time':datetime.strptime(i[1],'%H:%M').time(),'airline':i[2],'flight':i[3],'status':i[4]} for i in list_]
    for i in list_:
        if ":" in i.get('status',''):
            status,status_time = i.get('status').split(' ')
            i.update({'status':status,'status_time':datetime.strptime(status_time,'%H:%M').time()})
    return list_

In [8]:
#date Format: Y-M-D
x = airport_TUN(type_='departures',date='2019-12-10')

http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmairport=tunis&frmday=10&frmmonth=12&frmacty=2019&frmhour=0
http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmairport=tozeur&frmday=10&frmmonth=12&frmacty=2019&frmhour=0
http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmairport=djerba&frmday=10&frmmonth=12&frmacty=2019&frmhour=0
http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmairport=tabarka&frmday=10&frmmonth=12&frmacty=2019&frmhour=0
http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmairport=sfax&frmday=10&frmmonth=12&frmacty=2019&frmhour=0
http://www.oaca.nat.tn/newapp/hvdynt/resultatsvolsfrcall2.php?change=1&startRow=1&frmmvtCod=D&frmaeropVil=-1&frmairport=gafsa&frmday=10&frmmon

In [9]:
x

[{'Origine': 'ALGER',
  'time': datetime.time(9, 25),
  'airline': 'TUNISAIR',
  'flight': 'TU338',
  'status': 'DECOLLE',
  'status_time': datetime.time(10, 0)},
 {'Origine': 'ALGER',
  'time': datetime.time(9, 30),
  'airline': 'AIR ALGERIE',
  'flight': 'AH4001',
  'status': 'DECOLLE',
  'status_time': datetime.time(9, 33)},
 {'Origine': 'BAMAKO',
  'time': datetime.time(15, 20),
  'airline': 'TUNISAIR',
  'flight': 'TU613',
  'status': ' '},
 {'Origine': 'BELGRADE',
  'time': datetime.time(9, 35),
  'airline': 'TUNISAIR',
  'flight': 'TU226',
  'status': 'DECOLLE',
  'status_time': datetime.time(9, 58)},
 {'Origine': 'BENGHAZI',
  'time': datetime.time(3, 0),
  'airline': 'LYBIAN AIRLINES',
  'flight': 'LN307',
  'status': 'DECOLLE',
  'status_time': datetime.time(4, 22)},
 {'Origine': 'BENGHAZI',
  'time': datetime.time(12, 0),
  'airline': 'BURAQ AIR',
  'flight': 'UZ142',
  'status': 'RETARDE',
  'status_time': datetime.time(12, 40)},
 {'Origine': 'BENGHAZI',
  'time': datetime.